In [12]:
import numpy as np
from copy import deepcopy
import random

In [93]:
t_player_starts = [2,7]
st_time_labels = ['S1','S2']
traffic_lanes = ['T1','T2']

t_lights_on = 2

T1_on_frq = 5 # every 5 minutes
T2_on_frq = 30 # every 30 minutes

st_to_mid = 10 
mid_to_end = 10 

n_days = 180
activate_bayes_prb = 30
n_mins = 120

In [47]:
operate_times = lambda on_freq : np.arange(start=0,stop=n_mins, step=on_freq)

T1_start_end = np.concatenate(
    (operate_times(T1_on_frq).reshape((-1,1)),
    operate_times(T1_on_frq).reshape((-1,1))+2),
    axis=1
    )

T2_start_end = np.concatenate(
    (operate_times(T2_on_frq).reshape((-1,1)),
    operate_times(T2_on_frq).reshape((-1,1))+2),
    axis=1
    )


traffic_lane_st_end = {'T1':T1_start_end, 'T2':T2_start_end}

In [94]:
greater_lighton = (T1_start_end[:,0]<=12).reshape((-1,1))
less_lighton = (T1_start_end[:,1]>=12).reshape((-1,1))
combined_conditions = np.concatenate((greater_lighton,less_lighton), axis=1)
st, en = T1_start_end[np.all(combined_conditions, axis=1)][0]


def get_waiting_time(t_light_st_end, t_mid):
    greater_lighton = (t_light_st_end[:,0]<=t_mid).reshape((-1,1))
    less_lighton = (t_light_st_end[:,1]>=t_mid).reshape((-1,1))
    combined_conditions = np.concatenate((greater_lighton,less_lighton), axis=1)
    traffic_times = t_light_st_end[np.all(combined_conditions, axis=1)]

    if len(traffic_times)>0:
        traffic_start, traffic_end = traffic_times[0]

        wait_time = traffic_end-t_mid

        if wait_time==0:
            return 1
        return wait_time

    else:
        return 0

In [96]:
count_tracker = {
    'T1S1':0,
    'T1S2':0,
    '~T1S1':0,
    '~T1S2':0,
    'T2S1':0,
    'T2S2':0,
    '~T2S1':0,
    '~T2S2':0,
    'T1':0,
    'T2':0,
    '~T1':0,
    '~T2':0,
    'S1':0,
    'S2':0
}

posterior_prob = lambda st_label: (count_tracker[f'~T1{st_label}']/count_tracker[f'{st_label}'],
                                   count_tracker[f'~T2{st_label}']/count_tracker[f'{st_label}'])

daywise_record = {}


for day in range(1,n_days+1):
    st_time = random.sample(t_player_starts, k=1)[0]
    st_label = st_time_labels[t_player_starts.index(st_time)]
    count_tracker[st_label]+=1

    print(f'starting at {st_time}nd/th minute')
    curr_place = 'start'
    t_taken = 0
    t_to_mid = st_time+st_to_mid
    print(curr_place)
    
    
    while t_taken<=n_mins:
        if day<=activate_bayes_prb:
            chosen_lane = random.sample(traffic_lanes, k=1)[0]
        else:
            chosen_lane = random.choices(population=traffic_lanes, weights=posterior_prob(st_label))[0]

        t_taken+=t_to_mid
        curr_place = 'middle'
        print(curr_place)

        traf_light_time = get_waiting_time(traffic_lane_st_end[chosen_lane], t_taken)
        t_taken+=traf_light_time

        
        if traf_light_time==0:
            mod_chosen_lane = f'~{chosen_lane}'
        else:
            mod_chosen_lane = chosen_lane

        count_tracker[f'{mod_chosen_lane}{st_label}']+=1
        count_tracker[mod_chosen_lane]+=1
        

        t_taken+=st_to_mid*3
        curr_place = 'start'
        print(curr_place)


        
        
        

        
        
        



        
        

starting at 7nd/th minute
start
middle
start
middle
start
middle
start
starting at 7nd/th minute
start
middle
start
middle
start
middle
start
starting at 7nd/th minute
start
middle
start
middle
start
middle
start
starting at 7nd/th minute
start
middle
start
middle
start
middle
start
starting at 7nd/th minute
start
middle
start
middle
start
middle
start
starting at 2nd/th minute
start
middle
start
middle
start
middle
start
starting at 2nd/th minute
start
middle
start
middle
start
middle
start
starting at 7nd/th minute
start
middle
start
middle
start
middle
start
starting at 7nd/th minute
start
middle
start
middle
start
middle
start
starting at 2nd/th minute
start
middle
start
middle
start
middle
start
starting at 2nd/th minute
start
middle
start
middle
start
middle
start
starting at 2nd/th minute
start
middle
start
middle
start
middle
start
starting at 7nd/th minute
start
middle
start
middle
start
middle
start
starting at 7nd/th minute
start
middle
start
middle
start
middle
start
starti

In [97]:
count_tracker

{'T1S1': 13,
 'T1S2': 22,
 '~T1S1': 7,
 '~T1S2': 7,
 'T2S1': 0,
 'T2S2': 0,
 '~T2S1': 226,
 '~T2S2': 265,
 'T1': 35,
 'T2': 0,
 '~T1': 14,
 '~T2': 491,
 'S1': 82,
 'S2': 98}